# Batch Correction GAN v0.1

This was a proof of concept in oder to gain expirience with the different data formats neccescarry to train a GAN from single cell data and the techniques neccescary to evaluate such a model. This is a very early working prototype built to accomplish three learning objectives:

* How to present expression data as an input to a GAN.
* Better understand the properties and dimsneions of the data types neccescarry for the projects implementation.
* Gain familiarity with keras and SciKitlearn.

The process used to filter such a low number of cells (Unrealistic QC) will have obscured any biologicvally relevant signal. But a very small dataset is neccescarry to prototype so the goal is to

* Feed in data 
* Remove Cell labels 
* Input to netork
* Reattatch cell labels to output and evalaute results. 



## Import statements

In [1]:
import numpy as np
import pandas as pd
import scanpy.api as sc
import anndata



import os
import scipy
from scipy import sparse
import sys

%matplotlib inline 
import matplotlib.pyplot as plt
import datetime as datetime

import tensorflow as tf
import keras
import tensorboard
from keras.layers import Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from umap import UMAP
import seaborn as sns 

sc.settings.verbosity = 3 
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures
sc.logging.print_versions()

/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/scanpy/api/__init__.py:7: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  FutureWarning,
Using TensorFlow backend.


scanpy==1.5.1 anndata==0.7.3 umap==0.4.4 numpy==1.18.1 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.23.1 statsmodels==0.11.1 python-igraph==0.8.2 louvain==0.7.0


## TensorBoard

In [2]:
log_dir = "./tmp/logs/"
summary_writer = tf.summary.create_file_writer(logdir=log_dir)
%load_ext tensorboard
%tensorboard --logdir ./tmp/logs/

Reusing TensorBoard on port 6006 (pid 16523), started 3 days, 22:07:15 ago. (Use '!kill 16523' to kill it.)

# Reading in already preprocessed input data
* Here we read in the data and scale it using the scikit learn MinMax scaler. The feature range is set to  [-1, 1] as a prerequisite for usage of the tanh activation function in the final layer of the generator network.


* The data is in the format of an already normalised and log transfomred raw exspression matrix. We read in in as an array using the numpy genfromtext to load the data in as an array. The matix is transposed so that the each row value contains a single exspression profile for an idividual sample.


* Fit the scaler and delete the gene labels. The data is now prepared for input to the GAN netowrk.





In [2]:
# Reading in the data
input_matrix = np.genfromtxt('expr_mat.csv', skip_header=1)

# Transpose and check the shape
input_matrix = input_matrix.T
input_matrix.shape

# Set the scaler for concistancy 
scaler = MinMaxScaler(feature_range=(-1, 1))

# Fit the scalar to the input array
scaler.fit(input_matrix)

# Transform the values for each expresion profile to the range [-1,1]
input_matrix = scaler.transform(input_matrix)

# Delete the gene labels. (genfromtxt dosesn interprut these anyways,
# but feeding a row of NANs to the network has undesirable consequences

input_matrix = np.delete(input_matrix, (0), axis=0)






KeyboardInterrupt: 

# Model input dimensions

* This entire model and code other than the preprocessing was a reworking of a GAN used for greyscale images. We do not need to utilise 3 dimensional inputs (although setting then has mininmal effect i woud think). The model would learn pretty quicky - but given the complexity of the data the model intends to capture, one less thing to learn could prove important. An adjustment may improve the performance of subsequent models.

In [4]:
# Model input dimensions

rows = 1
cols = 132
depth = 1

cell = (rows, cols, depth)
z_dim = 100 # noise vector size

# Defining the generator network

* This is an adaption of the generator used in the image model this code was repurposed from.


* Simple architecture 2 dense layers with a leaky Relu activaion function used on in the first hidden layer, tanh applied in the final layer. 


* The leaky Relu function and tanh appear to be very commonly used. The Leaky ReLU alows small positive gradients to prevent vanishing gradients during training that lead to model collapse (traditional ReLU maps any.


* The hyperbolic tangent activation function in the final layer is regarded as producing crisper images. Originally sigmoid was the go to activation function for the output of a generator, but tanh appears to be most used.



In [5]:
# Generator as a function. Returns the generator network model.

def build_generator(cell, z_dim) :
  
  # Defines the model
  model = Sequential()

  # Adds a dense layer of 64 neurons with input_dim equal to z_dim
  model.add(Dense(64, input_dim = z_dim))

  # Apply Leaky ReLU activaion function 
  model.add(LeakyReLU(alpha=0.01))

  # Adds another fully connected layer - output layer tanh activation function
  model.add(Dense(1 * 132 * 1, activation = 'tanh'))

  model.add(Reshape(cell))

  return model

# Discriminator network

* Very sparse architecture - a single 128-neuron dense layer with leakyReLU activtion.


* Sigmoid in the final layer - The output of this networ is effectively the probability the network has of an input (single exspression profile being "real - 1" as oposed to "generated - 0".

In [6]:
# Defining the Discriminator network

def build_discriminator(cell) :

  # Defines the model
  model = Sequential()
  
 # Flatten - (deprecited in suqsequent models)
  model.add(Flatten(input_shape = cell))

 # Adds a dense layer of 128 neurons
  model.add(Dense(128))

 # Applys Leaky ReLU activaion function 
  model.add(LeakyReLU(alpha=0.01))

 # Applys sigmoid in the last layer. We want a probability value 
 # IE a probaility value in range [0,1].
  model.add(Dense(1, activation = 'sigmoid'))

  return model

# Compiling the model
* Connect the two models, compile the discriminator and the generator.


* Set the discrimator weights to untrainable for the connected model. We update the generator independantly using the output of the discrimator. This is a cental requirement for the model to leanr. Bothe networks are updated independantly in subsequent steps.


* Both models are compiled with the Adam optimiser - the most recomended for use with the vanilla GAN architecture. The loss function is binary cross entropy. This is because there are two classes being acessed. Real exspression profiles (single cells) and the generated exspression profiles or "cells" from the generator network.

In [7]:
# Build the GAn by joining the two networks
def build_gan(generator, discrimator) :

  model = Sequential()

  model.add(generator)
  model.add(discrimator)

  return model

# Build and compile the discrimator newtwork 
discrimator = build_discriminator(cell)
discrimator.compile(loss = 'binary_crossentropy',
                    optimizer = Adam(),
                    metrics = ['accuracy'])

# Build the generator network
generator = build_generator(cell, z_dim)
discrimator.trainable = False

# Build the whole model and compile
gan = build_gan(generator, discrimator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# Generating Cells

In [8]:
def generate_cells(number_cells) :
    
    # Draws from random normal.
    z = np.random.normal(0, 1, (1, z_dim))
    
    # predcts using enerator network.
    gen_cell = generator.predict(z)
    
    # convertts back to a 2d array.
    gen_cell = np.squeeze(gen_cell, axis=-1)
    
    # Transform back from MinMax scaling between -1 and 1 (necesscary for tanh activation function) 
    gen_cell = scaler.inverse_transform(gen_cell[0])


   # Repeat of above procedure inside a loop. 
    for cell in range(num_cells -1 ):
    
            z = np.random.normal(0, 1, (1, z_dim))
    
            gen_cells_tmp = generator.predict(z)
            
            gen_cells_tmp = np.squeeze(gen_cells_tmp, axis=-1)
    
            gen_cells_tmp = scaler.inverse_transform(gen_cells_tmp[0])
        
          #  Appends subsequent generated cells to form an array of generated cells
            gen_cell = np.append(gen_cell, gen_cells_tmp, axis=0)
    
    # Converts array to matrice object
    gen_cell = np.asmatrix(gen_cell)
    
    # converts matrice to pandas dataframe
    gen_cell = pd.DataFrame(gen_cell)
    
    # Loop to create cell names (theres deinitely a better way to do this)
    cell_names = []
    for i in range(num_cells ):
        cell_names.append("cell-{}".format(i +1))
        
        
    # Creates final dataframe with gene names and cell IDs
    gen_cell = pd.DataFrame(data=gen_cell.values, columns=exp_mat.columns, index = cell_names)
    
    return gen_cell


## Training the Model

In [9]:
losses = []
accuracies = []
iteration_checkpoints = []
Generated_cells = []



def train(iterations, batch_size, sample_interval, generate_cells_every, num_cells) :

  X_train =  input_matrix
  X_train = np.expand_dims(X_train, axis = 1)

  real = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for iteration in range(iterations) :

    idx = np.random.randint(X_train.shape[0], size = batch_size)
    real_cell = X_train[idx]
    real_cell = np.expand_dims(real_cell, axis = -1)

    z = np.random.normal(0, 1, (batch_size, z_dim))
    gen_cell = generator.predict(z)

    d_loss_real = discrimator.train_on_batch(real_cell, real)
    d_loss_fake = discrimator.train_on_batch(gen_cell, fake)
    d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)


    z = np.random.normal(0,1, (batch_size, 100))
    gen_imgs = generator.predict(z)
    g_loss = gan.train_on_batch(z, real)
    
    with summary_writer.as_default():
      tf.summary.scalar('d_loss', data=d_loss, step=iteration+1)
      tf.summary.scalar('g_loss', data=g_loss, step=iteration+1)
      tf.summary.scalar('accuracy', data=accuracy, step=iteration+1)


    if (iteration + 1) % sample_interval == 0:

      losses.append((d_loss, g_loss))
      accuracies.append(100.0 * accuracy)
      iteration_checkpoints.append(iteration + 1)

      print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" %
            (iteration + 1, d_loss, 100.0 * accuracy, g_loss))
        
    if (iteration +1) % generate_cells_every == 0:
        gen_cell = generate_cells(500)
        Generated_cells.append(gen_cell)
      

In [10]:
# running the model

# Setting hyper parameters
iterations = 25000
batch_size = 32
sample_interval = 1000
generate_cells_every = 5000
num_cells = 2000

train(iterations, batch_size, sample_interval, generate_cells_every, num_cells)

/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1000 [D loss: 0.571343, acc.: 75.00%] [G loss: 0.962245]
2000 [D loss: 0.431192, acc.: 81.25%] [G loss: 1.335050]
3000 [D loss: 0.573736, acc.: 73.44%] [G loss: 1.119047]
4000 [D loss: 0.611626, acc.: 62.50%] [G loss: 0.859219]
5000 [D loss: 0.738263, acc.: 67.19%] [G loss: 0.994808]


/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


6000 [D loss: 0.652020, acc.: 65.62%] [G loss: 1.022383]
7000 [D loss: 0.563567, acc.: 73.44%] [G loss: 1.379586]
8000 [D loss: 0.618601, acc.: 75.00%] [G loss: 1.198212]
9000 [D loss: 0.669087, acc.: 67.19%] [G loss: 1.060619]
10000 [D loss: 0.805287, acc.: 48.44%] [G loss: 0.948251]


/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


11000 [D loss: 0.696568, acc.: 60.94%] [G loss: 0.903237]
12000 [D loss: 0.731055, acc.: 54.69%] [G loss: 0.900254]
13000 [D loss: 0.601680, acc.: 62.50%] [G loss: 1.123104]
14000 [D loss: 0.757820, acc.: 54.69%] [G loss: 1.015571]
15000 [D loss: 0.665162, acc.: 67.19%] [G loss: 1.061366]


/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


16000 [D loss: 0.618348, acc.: 73.44%] [G loss: 1.016900]
17000 [D loss: 0.635922, acc.: 67.19%] [G loss: 0.997865]
18000 [D loss: 0.643113, acc.: 73.44%] [G loss: 1.262496]
19000 [D loss: 0.549346, acc.: 67.19%] [G loss: 0.932375]
20000 [D loss: 0.785000, acc.: 56.25%] [G loss: 0.980000]


/home/emcandrew/anaconda3/envs/tf_latest/lib/python3.7/site-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


21000 [D loss: 0.677545, acc.: 68.75%] [G loss: 1.174806]
22000 [D loss: 0.564325, acc.: 75.00%] [G loss: 0.979533]
23000 [D loss: 0.767626, acc.: 56.25%] [G loss: 0.973624]
24000 [D loss: 0.627756, acc.: 68.75%] [G loss: 1.338471]
25000 [D loss: 0.647126, acc.: 67.19%] [G loss: 1.055729]


In [11]:
Generated_cells[3]

,RPL11,SH3BGRL3,CD52,YBX1,RPS8,RPL5,S100A6,S100A4,RPS27,RPS7,...,RPS5,RPS4Y1,RPL3,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-CO3,MT-ND4,MT-CYB
cell-1,12.179202,14.615154,6.819412,15.106809,15.007879,15.781878,12.974711,12.741541,11.937942,13.689269,...,15.299510,6.581507,15.085269,13.304735,8.203832,14.095079,8.901382,11.269152,13.245955,10.941316
cell-2,10.630086,11.695746,11.218423,10.617737,11.903196,11.030147,10.697300,11.693042,10.900956,10.954415,...,12.184773,11.927264,12.573592,11.485777,10.538776,12.454388,10.193117,11.028885,10.619709,12.759210
cell-3,13.127371,14.202353,13.778810,9.666643,11.869294,12.924750,14.364780,10.741137,11.483773,11.140223,...,10.456700,12.482939,10.554552,6.896605,8.509381,11.288248,8.108074,10.329692,6.727996,12.991662
cell-4,14.355737,13.353405,14.177702,11.526878,15.674941,13.652715,13.631921,13.960871,11.392135,11.692746,...,13.763873,14.823050,13.635478,11.270316,10.190308,13.701278,10.143374,12.620340,10.768576,14.821025
cell-5,11.661810,11.499873,16.158426,15.826394,15.869644,15.687109,13.427846,12.720136,9.816346,14.072965,...,13.153356,15.295110,13.054567,6.233768,11.230963,14.109984,9.045271,12.423289,5.423950,15.392165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell-1996,11.516042,12.555659,11.275374,11.046193,11.381961,12.411295,13.710873,13.578465,10.724709,12.511080,...,11.840272,10.520197,11.867499,9.782765,9.813247,11.098029,8.103568,10.655293,10.172189,11.576513
cell-1997,13.635818,13.367567,2.521033,10.135159,14.443000,15.582382,13.052319,11.666033,14.003333,10.388412,...,11.413355,6.055169,13.826507,14.028158,9.994793,8.424507,11.655804,9.462490,12.041458,14.673732
cell-1998,14.825900,14.449038,15.649012,12.791083,15.642417,12.098851,7.581923,12.938647,12.125299,12.096626,...,14.288370,13.167167,14.676620,8.335036,9.553003,14.316132,9.590995,12.173903,9.648971,12.549890
cell-1999,9.501047,13.876179,11.789561,15.887345,15.311871,16.278994,12.023386,13.504231,10.719092,14.893492,...,15.218176,7.258925,14.163074,7.503932,10.044499,13.262273,5.847491,9.632274,10.953220,13.505488


In [12]:
output = Generated_cells[4]
output.to_csv('gen_cells.csv', sep='\t', index=True)

In [14]:
Generated_cells[4]

,RPL11,SH3BGRL3,CD52,YBX1,RPS8,RPL5,S100A6,S100A4,RPS27,RPS7,...,RPS5,RPS4Y1,RPL3,MT-ND1,MT-ND2,MT-CO1,MT-CO2,MT-CO3,MT-ND4,MT-CYB
cell-1,15.551950,12.227909,16.158991,15.582972,14.396317,10.645203,0.130422,0.000342,12.058917,14.507019,...,11.055149,14.647159,14.271068,14.821308,13.131840,13.500790,3.827966,11.457069,7.022162,14.846881
cell-2,14.715731,12.890346,15.349323,12.432402,12.711630,7.723995,7.142393,0.612229,11.454438,13.318100,...,11.573077,14.841235,12.469222,12.757236,14.201632,13.818088,4.853075,12.765859,9.527865,14.111885
cell-3,11.181349,10.896016,13.186731,14.302183,15.536703,7.483315,14.490423,15.555995,7.860258,13.672316,...,13.968018,3.505657,13.115487,12.970660,11.433341,12.487910,10.470879,12.506147,10.409985,5.823205
cell-4,9.915999,8.824220,13.116476,14.283776,12.776115,10.957479,5.470241,13.891963,9.619759,9.932469,...,12.805171,14.705087,11.667938,9.932622,11.331962,9.329876,6.586185,9.411663,9.452892,8.579746
cell-5,11.574150,10.906353,12.993999,12.712704,15.582344,11.100823,12.787998,10.908566,11.000291,11.536145,...,13.099494,9.482021,13.208686,13.369393,12.704232,13.161552,9.206448,11.183058,9.557903,11.462231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
cell-1996,8.844520,10.446237,8.732923,11.534192,11.112865,10.492652,11.333441,12.821640,10.321572,11.710045,...,12.594736,11.166951,10.401993,10.717392,11.050464,10.917593,9.205895,9.921863,10.105810,11.963284
cell-1997,7.619344,3.649522,6.855786,13.702393,7.639131,12.327373,6.145095,10.298156,12.133239,13.049295,...,10.698046,10.122531,12.654674,7.493376,10.570169,11.284751,4.798995,9.665277,9.674637,9.645466
cell-1998,14.885834,12.481987,15.577119,15.633497,15.833282,11.048656,9.589485,2.298280,10.785499,12.844518,...,12.921499,13.367822,13.594203,13.855461,13.454860,12.880948,9.006853,11.259155,10.840840,13.470925
cell-1999,12.230768,13.500591,11.371217,13.203501,13.256593,10.640863,13.114392,13.787264,10.403722,10.939392,...,12.345242,7.885275,10.427363,9.998962,11.465369,11.191114,7.722419,10.390137,8.381648,10.970373
